https://adventofcode.com/2019/day/17

In [1]:
import io
import networkx as nx
from intcode_computer import Intcode

In [2]:
with open('data/17.txt') as fh:
    data = fh.read().strip()
inp = [int(x) for x in data.split(',')]

In [3]:
ic = Intcode(inp.copy())
asc = []
while True:
    try:
        asc.append(ic.step())
    except StopIteration as e:
        print(e)
        break

Program has halted


In [4]:
len(asc)

2060

In [5]:
stream = io.StringIO(''.join(chr(x) for x in asc if x is not None))
stream.seek(0)
txt = stream.read()

In [6]:
print(txt)

#######..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#..................................
#.....#.......................#####......
#.....#.......................#...#......
#.....#.......................#...#......
#.....#.......................#...#......
###########.................###########..
......#...#.................#.#...#...#..
......#######...........#######...#...#..
..........#.#...........#...#.....#...#..
..........#.#...........#...#.....#...#..
..........#.#...........#...#.....#...#..
..........#######.......#.#############..
............#...#.......#.#.#.....#......
............#############.#.#.....######^
................#.........#.#............
................#.....#######............
................#.....#...#..............
................#...#######..............
................#...#.#...........

In [7]:
scaffold = {}
for y, line in enumerate(txt.split('\n')):
    line = line.strip()
    for x, c in enumerate(line):
        if c in '#^v<>':
            scaffold[(x, y)] = c

In [8]:
len(scaffold)

262

In [9]:
def listnabes(pos):
    x, y = pos
    return [(x, y+1), (x+1, y), (x, y-1), (x-1, y)]

In [10]:
G = nx.Graph()

In [11]:
for pos, c in scaffold.items():
    G.add_node(pos, c=c)
    for nabe in listnabes(pos):
        if nabe in scaffold:
            G.add_node(nabe, c=scaffold[nabe])
            G.add_edge(pos, nabe)

In [12]:
len(G)

262

In [13]:
for node in G.nodes:
    nabes = list(nx.neighbors(G, node))
    if len(nabes) == 4:
        print(node, nabes)

(6, 10) [(6, 9), (5, 10), (6, 11), (7, 10)]
(30, 10) [(30, 9), (29, 10), (30, 11), (31, 10)]
(34, 10) [(34, 9), (33, 10), (34, 11), (35, 10)]
(10, 12) [(10, 11), (9, 12), (10, 13), (11, 12)]
(28, 12) [(28, 11), (27, 12), (28, 13), (29, 12)]
(12, 16) [(12, 15), (11, 16), (12, 17), (13, 16)]
(28, 16) [(28, 15), (27, 16), (28, 17), (29, 16)]
(34, 16) [(34, 15), (33, 16), (34, 17), (35, 16)]
(16, 18) [(16, 17), (15, 18), (16, 19), (17, 18)]
(26, 20) [(26, 19), (25, 20), (26, 21), (27, 20)]
(22, 22) [(22, 21), (21, 22), (22, 23), (23, 22)]
(16, 26) [(16, 25), (15, 26), (16, 27), (17, 26)]
(20, 26) [(20, 25), (19, 26), (20, 27), (21, 26)]
(18, 36) [(18, 35), (17, 36), (18, 37), (19, 36)]
(22, 38) [(22, 37), (21, 38), (22, 39), (23, 38)]


In [14]:
for node in G.nodes:
    nabes = list(nx.neighbors(G, node))
    if len(nabes) == 3:
        print(node, nabes)

In [15]:
for node in G.nodes:
    nabes = list(nx.neighbors(G, node))
    if len(nabes) == 1:
        print(node, nabes)

(40, 18) [(39, 18)]
(18, 32) [(18, 33)]


In [16]:
param_sum = 0
for node in G.nodes:
    nabes = list(nx.neighbors(G, node))
    if len(nabes) == 4:
        x, y = node
        param_sum += (x * y)
print(param_sum)

6052


## Part 2

DFS starting at robot.

In [17]:
n = next(iter(G.nodes))
type(n), n

(tuple, (0, 0))

In [18]:
G.nodes[(0,0)]

{'c': '#'}

In [19]:
for n in G.nodes:
    if G.nodes[n]['c'] in '^v<>':
        startnode = n
        drxn = G.nodes[n]['c']
        print(n, drxn)

(40, 18) ^


### DFS-DG (depth first search with delayed gratification)

In [25]:
firstnode = (40,18)
lastnode = (18,32)

frontier = [firstnode] # stack
visited = {firstnode}
log = []

intersections = set()
prev = firstnode


while frontier:
    node = frontier.pop()
    log.append(node)
    
    nabes = [x for x in nx.neighbors(G, node) if x != prev and (x not in visited or x in intersections)]
    if len(nabes) > 2:
        # this is an intersection
        intersections.add(node)
        nabes.sort(key = lambda x: len(list(nx.shortest_path(G, x, lastnode))))
    for nabe in nabes:
        frontier.append(nabe)
        visited.add(nabe)
    prev = node

In [26]:
len(visited), len(G)

(262, 262)

In [27]:
len(log)

280

In [28]:
log[-10:]

[(29, 10),
 (34, 11),
 (34, 12),
 (34, 13),
 (34, 14),
 (34, 10),
 (33, 10),
 (34, 15),
 (34, 16),
 (33, 16)]

Are there any jumps in the log?

In [29]:
it = iter(log)
a = next(it)
c = 0
for i, b in enumerate(it, 1):
    if b not in list(nx.neighbors(G, a)):
        c += 1
        print(i, 'jump', a, b)
    a = b
print(f"{c} jumps")

37 jump (32, 10) (30, 11)
43 jump (28, 10) (28, 13)
51 jump (32, 16) (28, 17)
61 jump (26, 16) (26, 21)
71 jump (24, 20) (22, 23)
87 jump (18, 26) (16, 25)
89 jump (16, 26) (16, 24)
112 jump (26, 12) (15, 18)
114 jump (16, 18) (14, 18)
181 jump (16, 18) (15, 26)
183 jump (16, 26) (14, 26)
254 jump (18, 32) (20, 25)
256 jump (20, 26) (20, 24)
259 jump (20, 22) (19, 26)
261 jump (20, 26) (22, 22)
263 jump (21, 22) (26, 20)
265 jump (25, 20) (28, 16)
267 jump (27, 16) (28, 12)
269 jump (27, 12) (30, 10)
271 jump (29, 10) (34, 11)
275 jump (34, 14) (34, 10)
277 jump (33, 10) (34, 15)
22 jumps


In [88]:
intersections

{(6, 10),
 (10, 12),
 (12, 16),
 (16, 18),
 (16, 26),
 (18, 36),
 (20, 26),
 (22, 22),
 (22, 38),
 (26, 20),
 (28, 12),
 (28, 16),
 (30, 10),
 (34, 10),
 (34, 16)}

In [143]:
log[20:32]

[(36, 10),
 (35, 10),
 (34, 10),
 (34, 11),
 (34, 12),
 (34, 13),
 (34, 14),
 (34, 9),
 (34, 10),
 (34, 8),
 (34, 7),
 (34, 6)]

Now fill in the jumps in the log.

In [152]:
rlog = list(reversed(log))

track = [rlog.pop()]
track

[(40, 18)]

In [153]:
while rlog:
    prev = track[-1]
    node = rlog.pop()
    if node in list(nx.neighbors(G, prev)):
        track.append(node)
    else:
        track.extend(nx.shortest_path(G, prev, node)[1:])

In [154]:
len(track)

374

In [155]:
track[-20:]

[(30, 8),
 (30, 7),
 (30, 6),
 (31, 6),
 (32, 6),
 (33, 6),
 (34, 6),
 (34, 7),
 (34, 8),
 (34, 9),
 (34, 10),
 (34, 11),
 (34, 12),
 (34, 13),
 (34, 14),
 (34, 15),
 (34, 16),
 (33, 16),
 (34, 16),
 (34, 15)]

In [156]:
class BotTape:
    def __init__(self, track=track, drxn=drxn):
        self.trackit = iter(track)
        self.drxn = drxn
        self.tape = []
        self.pos = next(self.trackit)
        self.counter = 0
    
    def run(self):
        for newpos in self.trackit:
            self.move(newpos)
        if self.counter:
            self.tape.append(str(self.counter))
        
    def move(self, newpos):
        if newpos == self.pos:
            raise BotTapeError(f"Track repeated {newpos}")
        drxn, drcmds = self.getdrxn(newpos)
        self.drxn = drxn
        if drcmds:
            if self.counter:
                self.tape.append(str(self.counter))
                self.counter = 0
            self.tape.extend(drcmds)
        self.counter += 1
        self.pos = newpos
    
    def getdrxn(self, newpos):
        (x1, y1) = self.pos
        (x2, y2) = newpos
        
        if x2 > x1:
            drxn = '>'
        elif x2 < x1:
            drxn = '<'
        elif y2 > y1:
            drxn = 'v'
        elif y2 < y1:
            drxn = '^'
        else:
            raise BotTapeError("Can't happen")
        
        if drxn == self.drxn:
            return (drxn, [])
        
        turns = {
            ('^', '>'): ['R'],
            ('^', 'v'): ['R', '0', 'R'],
            ('^', '<'): ['L'],
            ('>', 'v'): ['R'],
            ('>', '<'): ['R', '0', 'R'],
            ('>', '^'): ['L'],
            ('v', '<'): ['R'],
            ('v', '^'): ['R', '0', 'R'],
            ('v', '>'): ['L'],
            ('<', '^'): ['R'],
            ('<', '>'): ['R', '0', 'R'],
            ('<', 'v'): ['L']
        }
        return (drxn, turns[self.drxn, drxn])

class BotTapeError(Exception):
    pass  

In [157]:
bt = BotTape()
bt.run()

In [158]:
print(''.join(bt.tape))

L6R2R4L6L4L4R0R4L4L2R2L4L4R0R4L4R2L2R4L4R2L2R4R2R2R0R8L4L6R6L6R2L6R10R6R10L4R6R0R4R2R6L6R4R6L8R8L6R2R0R2L6R12R2R4R0R6R4R2R2R0R2L6R2R4R0R10R10R6R10L4R2L2R4L4R10L4L4R2L2R4L4R2L4L1R0R1L2R2L4R4R10R1R0R1L1


In [159]:
len(bt.tape)

192

In [160]:
s = 'R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2'.replace(',', '')
s

'R8R8R4R4R8L6L2R4R4R8R8R8L6L2'

In [161]:
def repeated_prefixes(s, minlen, maxlen, stride):
    L = []
    slen = len(s)
    for i in range(minlen, maxlen+1, stride):
        prefix, remainder = s[:i], s[i:]
        while prefix in remainder:
            remainder = remainder.replace(prefix, '', 1)
        if len(prefix) + len(remainder) < slen:
            L.append((prefix, remainder))
    L.sort(key = lambda x: x[1])
    return L

In [162]:
L = repeated_prefixes(s, 4, 7, 2)
L

[('R8R8', 'R4R4R8L6L2R4R4R8L6L2')]

In [163]:
t = L[0][1]
M = repeated_prefixes(t, 4, 7, 2)
M

[('R4R4R8', 'L6L2L6L2'), ('R4R4', 'R8L6L2R8L6L2')]

In [164]:
u = M[0][1]
N = repeated_prefixes(u, 4, 7, 2)
N

[('L6L2', '')]

In [165]:
for (A, rA) in repeated_prefixes(s, 4, 7, 2):
    for (B, rB) in repeated_prefixes(rA, 4, 7, 2):
        for (C, rC) in repeated_prefixes(rB, 4, 7, 2):
            if not rC:
                print(A, B, C)

R8R8 R4R4R8 L6L2
R8R8 R4R4 R8L6L2


In [166]:
thetape = ''.join(bt.tape)
print(thetape)

L6R2R4L6L4L4R0R4L4L2R2L4L4R0R4L4R2L2R4L4R2L2R4R2R2R0R8L4L6R6L6R2L6R10R6R10L4R6R0R4R2R6L6R4R6L8R8L6R2R0R2L6R12R2R4R0R6R4R2R2R0R2L6R2R4R0R10R10R6R10L4R2L2R4L4R10L4L4R2L2R4L4R2L4L1R0R1L2R2L4R4R10R1R0R1L1


In [171]:
for (A, rA) in repeated_prefixes(thetape, 4, 15, 2):
    for (B, rB) in repeated_prefixes(rA, 4, 15, 2):
        for (C, rC) in repeated_prefixes(rB, 4, 15, 2):
            if not rC:
                print(A, B, C)

In [172]:
repeated_prefixes(thetape, 4, 15, 2)

[('L6R2R4',
  'L6L4L4R0R4L4L2R2L4L4R0R4L4R2L2R4L4R2L2R4R2R2R0R8L4L6R6L6R2L6R10R6R10L4R6R0R4R2R6L6R4R6L8R8L6R2R0R2L6R12R2R4R0R6R4R2R2R0R2R0R10R10R6R10L4R2L2R4L4R10L4L4R2L2R4L4R2L4L1R0R1L2R2L4R4R10R1R0R1L1'),
 ('L6R2',
  'R4L6L4L4R0R4L4L2R2L4L4R0R4L4R2L2R4L4R2L2R4R2R2R0R8L4L6R6L6R10R6R10L4R6R0R4R2R6L6R4R6L8R8R0R2L6R12R2R4R0R6R4R2R2R0R2R4R0R10R10R6R10L4R2L2R4L4R10L4L4R2L2R4L4R2L4L1R0R1L2R2L4R4R10R1R0R1L1')]